In [1]:
!pip install ray==2.10.0
!pip install autogluon.tabular
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.40.0
    Uninstalling ray-2.40.0:
      Successfully uninstalled ray-2.40.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/alter-data/credit(120).csv
/kaggle/input/alter-data/alter_data.csv


In [3]:
auto_df_train = pd.read_csv('/kaggle/input/alter-data/alter_data.csv')
auto_df_test = pd.read_csv('/kaggle/input/alter-data/credit(120).csv')

display(auto_df_train.head())
display(auto_df_test.head())

,소비 빈도 및 활동성,소비 금액 및 규모,연체 가능성 및 신용 리스크,서비스 이용 다양성,남성,여성,20대,30대,40대,50대,60대,70대,연체여부
0,0.641394,0.768399,0.383721,0.409465,True,False,True,False,False,False,False,False,1.0
1,0.586476,0.652896,0.372093,0.454870,True,False,False,True,False,False,False,False,1.0
2,0.460871,0.612550,0.348837,0.438317,True,False,False,True,False,False,False,False,1.0
3,0.497903,0.563611,0.279070,0.426953,True,False,False,False,True,False,False,False,0.0
4,0.515722,0.523879,0.267442,0.420061,True,False,False,False,True,False,False,False,0.0


,소비 빈도 및 활동성,소비 금액 및 규모,연체 가능성 및 신용 리스크,서비스 이용 다양성,남성,여성,20대,30대,40대,50대,60대,70대,회원여부_연체
0,0.384426,0.077502,0.5,0.25,0,1,0,0,1,0,0,0,1
1,0.265004,0.042410,1.0,0.25,0,1,1,0,0,0,0,0,1
2,0.076045,0.038398,0.5,0.25,0,1,0,0,0,1,0,0,1
3,0.119652,0.045946,0.5,0.25,0,1,0,1,0,0,0,0,1
4,0.168577,0.040089,0.5,0.25,1,0,0,0,0,0,1,0,1


In [4]:
auto_df_train.iloc[:, 4:]= auto_df_train.iloc[:, 4:].astype(int)
auto_df_train.head()

,소비 빈도 및 활동성,소비 금액 및 규모,연체 가능성 및 신용 리스크,서비스 이용 다양성,남성,여성,20대,30대,40대,50대,60대,70대,연체여부
0,0.641394,0.768399,0.383721,0.409465,1,0,1,0,0,0,0,0,1.0
1,0.586476,0.652896,0.372093,0.454870,1,0,0,1,0,0,0,0,1.0
2,0.460871,0.612550,0.348837,0.438317,1,0,0,1,0,0,0,0,1.0
3,0.497903,0.563611,0.279070,0.426953,1,0,0,0,1,0,0,0,0.0
4,0.515722,0.523879,0.267442,0.420061,1,0,0,0,1,0,0,0,0.0


In [5]:
auto_df_test.rename(columns={'회원여부_연체': '연체여부'}, inplace=True)
auto_df_test.head()

,소비 빈도 및 활동성,소비 금액 및 규모,연체 가능성 및 신용 리스크,서비스 이용 다양성,남성,여성,20대,30대,40대,50대,60대,70대,연체여부
0,0.384426,0.077502,0.5,0.25,0,1,0,0,1,0,0,0,1
1,0.265004,0.042410,1.0,0.25,0,1,1,0,0,0,0,0,1
2,0.076045,0.038398,0.5,0.25,0,1,0,0,0,1,0,0,1
3,0.119652,0.045946,0.5,0.25,0,1,0,1,0,0,0,0,1
4,0.168577,0.040089,0.5,0.25,1,0,0,0,0,0,1,0,1


In [6]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='연체여부',
                            eval_metric='mcc',
                            problem_type='binary').fit(auto_df_train,
                                                       presets='best_quality',
                                                        time_limit=3600*8,
                                                       raise_on_no_models_fitted=False,
                                                    #    excluded_model_types=['KNN','XGBoost'],
                                                       ag_args_fit={'num_gpus': 1},
                                                       auto_stack=True,
#                                                        num_bag_folds=5,
#                                                        num_stack_levels=2,
#                                                        num_bag_sets = 1,
                                                      )
results = predictor.fit_summary(verbosity=1)
 

No path specified. Models will be saved in: "AutogluonModels/ag-20250106_060220"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.08 GB / 31.35 GB (96.0%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

(_ray_fit pid=3923) [1000]	valid_set's binary_logloss: 0.482116	valid_set's mcc: 0.531678


(_ray_fit pid=3991) 	Training S1F3 with GPU, note that this may negatively impact model quality compared to CPU training. [repeated 2x across cluster]
(_ray_fit pid=4026) 	Training S1F4 with GPU, note that this may negatively impact model quality compared to CPU training.
(_ray_fit pid=4052) 	Training S1F5 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=4026) [1000]	valid_set's binary_logloss: 0.477215	valid_set's mcc: 0.55054 [repeated 3x across cluster]


(_ray_fit pid=4096) 	Training S1F6 with GPU, note that this may negatively impact model quality compared to CPU training.


...

(_ray_fit pid=11853) [2000]	valid_set's binary_logloss: 0.480855	valid_set's mcc: 0.531987


(_ray_fit pid=11959) 	Training S1F6 with GPU, note that this may negatively impact model quality compared to CPU training.


(_ray_fit pid=11959) [1000]	valid_set's binary_logloss: 0.472793	valid_set's mcc: 0.550703


(_ray_fit pid=11995) 	Training S1F7 with GPU, note that this may negatively impact model quality compared to CPU training.
(_dystack pid=173) 	0.5388	 = Validation score   (mcc)
(_dystack pid=173) 	31.77s	 = Training   runtime
(_dystack pid=173) 	1.35s	 = Validation runtime
(_dystack pid=173) Fitting model: NeuralNetTorch_r158_BAG_L1 ... Training model for up to 326.51s of the 2726.23s of remaining time.
(_ray_fit pid=12029) 	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
(_dystack pid=173) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2.0 workers, per: cpus=1, gpus=1, memory=0.03%)
(_ray_fit pid=12132) 	Ran out of time, stopping training early. (Stopping on epoch 76)
(_ray_fit pid=12166) 	Ran out of time, stopping training early. (Stopping on epoch 76)
(_ray_fit pid=12200) 	Ran out of time, stopping training early. (Stopping on epoch 76)
(_ray_fit pid=12234) 	Ran out of time, stopping traini

*** Summary of fit() ***
Estimated performance of each model:
                           model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0            WeightedEnsemble_L3   0.555835         mcc      52.445503  4827.681274                0.004437           5.946056            3       True        207
1                 XGBoost_BAG_L2   0.555816         mcc      52.271777  4799.566803                0.158778          15.030980            2       True        120
2             XGBoost_r22_BAG_L2   0.554833         mcc      52.279359  4798.582579                0.166360          14.046756            2       True        192
3             XGBoost_r89_BAG_L2   0.554782         mcc      52.287435  4799.701518                0.174437          15.165695            2       True        138
4             XGBoost_r98_BAG_L2   0.554763         mcc      52.282288  4806.704237                0.169289          22.168415  

In [7]:
# 모델 예측(라벨값 제외)
y_pred = predictor.predict(auto_df_test)

## 모델 평가
evaluation_results = predictor.evaluate(auto_df_test, silent=True)
evaluation_results

{'mcc': -0.04860246181385666,
 'accuracy': 0.714485,
 'balanced_accuracy': 0.48906528068901445,
 'roc_auc': 0.45328720091797225,
 'f1': 0.04169230597106768,
 'precision': 0.15393353574157836,
 'recall': 0.02411139184813026}